# SQLAlchemy

The SQLAlchemy models allows for the translation between OO languages like python and relational languages like SQL. It transforms python objects into database information, and the other way around. PSQLAlcheny can connect to many differnt types of sql databases, but all of them follow this standard. The standard specifies the functions that help you connect to a database, send queries, and get results. All the specifications are regulated by PEP 249.

In [4]:
!pip install sqlalchemy

/bin/sh: 1: pip: not found


For this topic we will practic on the `Buildings_Database.sqlite`. using the folowing code we can connect to the databse.

In [9]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///Buildings_Database.sqlite', echo=True)
connection = engine.connect()
print(connection, engine)

<sqlalchemy.engine.base.Connection object at 0x7f07ca3e6200> Engine(sqlite:///Buildings_Database.sqlite)


In [10]:
from sqlalchemy import inspect
inspector = inspect(engine)
inspector.get_table_names()

2023-04-19 21:49:17,045 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-19 21:49:17,046 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-04-19 21:49:17,046 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-19 21:49:17,051 INFO sqlalchemy.engine.Engine ROLLBACK


['Buildings']

!! other code in topic https://hyperskill.org/learn/step/12935 that i did not get working

## Data mapping

Mapping transforms SQL objects into Python ones. There are two main mapping types in SQLAlchemy. One is **classical** mapping, the other is **declarative** mapping. We will discuss these types below.

### Classical

Classical mapping refers to the configuration of **a mapped class** that was created with the `mapper()` function. We need to define a database table and the corresponding Python class separately to link them with `mapper()`. After that, all changes to the table and class made via SQLAlchemy are saved in your database. Classical mapping is a **base mapping system** provided by the ORM. Take a look at the snippet below:

````python
from sqlalchemy import Table, MetaData, Column, Integer, String
from sqlalchemy.orm import mapper

metadata = MetaData()

animals = Table('animals', metadata,
                Column('id', Integer, primary_key=True),
                Column('petname', String(30)),
                Column('age', Integer),
                Column('weight', Integer))


class Animals(object):
    def __init__(self, petname, age, weight):
        self.petname = petname
        self.age = age
        self.weight = weight


mapper(Animals, animals)
````

It however does not seem to work with SQLAlchemy 2.0. Must look up right topic

### Declarative

Declarative mapping is a concise version of classical mapping. We don't need to specify a class and a table separately, we can do it all in one class. Let's try to write a concise form of the previous snippet:

!! Had to be changed to match with **SQLAlchemy 2.0**: `from sqlalchemy.orm import declarative_base`

In [19]:
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String

Base = declarative_base()


class Animals(Base):
    __tablename__ = 'animals'

    id = Column(Integer, primary_key=True)
    petname = Column(String(30))
    age = Column(Integer)
    weight = Column(Integer)

In [20]:
Base.metadata.create_all(engine)

2023-04-19 22:03:05,653 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-19 22:03:05,654 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("animals")
2023-04-19 22:03:05,655 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-19 22:03:05,658 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("animals")
2023-04-19 22:03:05,659 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-19 22:03:05,660 INFO sqlalchemy.engine.Engine 
CREATE TABLE animals (
	id INTEGER NOT NULL, 
	petname VARCHAR(30), 
	age INTEGER, 
	weight INTEGER, 
	PRIMARY KEY (id)
)


2023-04-19 22:03:05,660 INFO sqlalchemy.engine.Engine [no key 0.00036s] ()
2023-04-19 22:03:05,672 INFO sqlalchemy.engine.Engine COMMIT


### Session
The `mapper()` function and declarative extensions are primary interfaces for ORM. Once our mappings are configured, we can proceed to the primary interface. It is also known as a session. It helps us to communicate with our database and ensures that all operations run smoothly. You can modify your database and save the changes during the session. To start one, you can use the following statement:

In [22]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()
session

In [23]:
animal_1 = Animals(petname='Timmy', age=2, weight=12)
animal_2 = Animals(petname='Tommy', age=3, weight=10)
animal_3 = Animals(petname='Kitty', age=4, weight=14)
session.add(animal_1)
session.add(animal_2)
session.add(animal_3)
session.commit()

2023-04-19 22:05:13,709 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-19 22:05:13,711 INFO sqlalchemy.engine.Engine INSERT INTO animals (petname, age, weight) VALUES (?, ?, ?), (?, ?, ?), (?, ?, ?) RETURNING id
2023-04-19 22:05:13,712 INFO sqlalchemy.engine.Engine [generated in 0.00006s (insertmanyvalues)] ('Timmy', 2, 12, 'Tommy', 3, 10, 'Kitty', 4, 14)
2023-04-19 22:05:13,717 INFO sqlalchemy.engine.Engine COMMIT


## querying and filtering

 A Query object will generate a SELECT statement for columns of our table and rename each column to a variable name as per PEP convention. Here's what it may look like. Once passed, it will create the animals_petname variable for the table called Animals with the petname column. After this, you'll be able to access the values in the corresponding column via this variable.

In [26]:
from sqlalchemy.orm import Query
query = Query(Animals)
query

In [27]:
print(query)

SELECT animals.id AS animals_id, animals.petname AS animals_petname, animals.age AS animals_age, animals.weight AS animals_weight 
FROM animals


In [28]:
from sqlalchemy.orm import Query, sessionmaker
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:", echo=True)
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

2023-04-19 22:10:48,612 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-19 22:10:48,613 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("animals")
2023-04-19 22:10:48,614 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-19 22:10:48,614 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("animals")
2023-04-19 22:10:48,615 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-19 22:10:48,615 INFO sqlalchemy.engine.Engine 
CREATE TABLE animals (
	id INTEGER NOT NULL, 
	petname VARCHAR(30), 
	age INTEGER, 
	weight INTEGER, 
	PRIMARY KEY (id)
)


2023-04-19 22:10:48,615 INFO sqlalchemy.engine.Engine [no key 0.00026s] ()
2023-04-19 22:10:48,616 INFO sqlalchemy.engine.Engine COMMIT


In [32]:
query = session.query(Animals)
print(query)

SELECT animals.id AS animals_id, animals.petname AS animals_petname, animals.age AS animals_age, animals.weight AS animals_weight 
FROM animals


In [33]:
session.add(Animals(petname="Billy", age=1, weight=8))
session.add(Animals(petname="Susan", age=4, weight=12))

session.commit()

2023-04-19 22:12:25,115 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-19 22:12:25,117 INFO sqlalchemy.engine.Engine INSERT INTO animals (petname, age, weight) VALUES (?, ?, ?), (?, ?, ?) RETURNING id
2023-04-19 22:12:25,118 INFO sqlalchemy.engine.Engine [generated in 0.00006s (insertmanyvalues)] ('Billy', 1, 8, 'Susan', 4, 12)
2023-04-19 22:12:25,118 INFO sqlalchemy.engine.Engine COMMIT


In [34]:
all_rows = query.all()

2023-04-19 22:12:35,887 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-19 22:12:35,889 INFO sqlalchemy.engine.Engine SELECT animals.id AS animals_id, animals.petname AS animals_petname, animals.age AS animals_age, animals.weight AS animals_weight 
FROM animals
2023-04-19 22:12:35,889 INFO sqlalchemy.engine.Engine [generated in 0.00041s] ()


In [35]:
for row in all_rows:
    print(type(row))  # <class '__main__.Animals'>

<class '__main__.Animals'>
<class '__main__.Animals'>


In [36]:
for row in all_rows:
    print(f"Pet name: {row.petname}, age: {row.age}, weight: {row.weight}")

Pet name: Billy, age: 1, weight: 8
Pet name: Susan, age: 4, weight: 12


In [37]:
query = session.query(Animals.petname)
print(query)

SELECT animals.petname AS animals_petname 
FROM animals


In [38]:
query = session.query(Animals.petname, Animals.age)
for petname, age in query:
    print(petname, age)

2023-04-19 22:13:16,940 INFO sqlalchemy.engine.Engine SELECT animals.petname AS animals_petname, animals.age AS animals_age 
FROM animals
2023-04-19 22:13:16,941 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ()
Billy 1
Susan 4


In [39]:
query = session.query(Animals)
query.count()

2023-04-19 22:13:39,335 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT animals.id AS animals_id, animals.petname AS animals_petname, animals.age AS animals_age, animals.weight AS animals_weight 
FROM animals) AS anon_1
2023-04-19 22:13:39,336 INFO sqlalchemy.engine.Engine [generated in 0.00083s] ()


2

In [40]:
query = session.query(Animals)
for row in query.filter(Animals.petname == "Billy"):
    print(row.petname, row.age, row.weight) # Billy 1 8

2023-04-19 22:13:53,456 INFO sqlalchemy.engine.Engine SELECT animals.id AS animals_id, animals.petname AS animals_petname, animals.age AS animals_age, animals.weight AS animals_weight 
FROM animals 
WHERE animals.petname = ?
2023-04-19 22:13:53,457 INFO sqlalchemy.engine.Engine [generated in 0.00146s] ('Billy',)
Billy 1 8


In [42]:
query = session.query(Animals.age, Animals.weight)

age_gr_than = Animals.age > 2
weight_eq_gr_than = Animals.weight >= 8

for age, weight in query.filter(age_gr_than, weight_eq_gr_than):
    print(f"Pet age: {age}, Pet weight: {weight}")

2023-04-19 22:14:30,344 INFO sqlalchemy.engine.Engine SELECT animals.age AS animals_age, animals.weight AS animals_weight 
FROM animals 
WHERE animals.age > ? AND animals.weight >= ?
2023-04-19 22:14:30,345 INFO sqlalchemy.engine.Engine [generated in 0.00108s] (2, 8)
Pet age: 4, Pet weight: 12
